In [12]:
import pandas as pd
import numpy as np
import pickle

In [13]:
train=pd.read_csv('final_data_train.csv')

In [14]:
test=pd.read_csv('final_data_test.csv')

In [15]:
# splitting the Data
x=train.drop(['Price'],axis=1)
y=train['Price']

In [16]:
x.drop(['Unnamed: 0'],axis=1,inplace=True)
x.head()

,Airline,Source,Destination,Route,Additional_Info,Duration,Total_Stops,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min
0,3,0,5,18,4,170,0,24,3,22,20,1,10
1,1,3,0,84,4,445,2,1,5,5,50,13,15
2,4,2,1,118,4,1140,2,9,6,9,25,4,25
3,3,3,0,91,4,325,1,12,5,18,5,23,30
4,3,0,5,29,4,285,1,1,3,16,50,21,35


In [17]:
y.head()

0     3897
1     7662
2    13882
3     6218
4    13302
Name: Price, dtype: int64

# Building Machine Learning Models

In [37]:
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 77)

In [21]:
print("The size of training input is", X_train.shape)
print("The size of training output is", y_train.shape)
print(50 *'*')
print("The size of testing input is", X_test.shape)
print("The size of testing output is", y_test.shape)

The size of training input is (8369, 13)
The size of training output is (8369,)
**************************************************
The size of testing input is (2093, 13)
The size of testing output is (2093,)


# K Neighbors Regressor

In [24]:
k_range = list(range(1, 30))
params = dict(n_neighbors = k_range)
knn_regressor = GridSearchCV(KNeighborsRegressor(), params, cv =10, scoring = 'neg_mean_squared_error')
knn_regressor.fit(X_train, y_train)


GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29]},
             scoring='neg_mean_squared_error')

In [25]:
y_train_pred =knn_regressor.predict(X_train)
y_test_pred =knn_regressor.predict(X_test) 

In [35]:
r2_score(y_train.values, y_train_pred)

0.8081424233033154

# Decision Tree Regressor

In [38]:
depth  =list(range(3,30))
param_grid =dict(max_depth =depth)
tree =GridSearchCV(DecisionTreeRegressor(),param_grid,cv =10)
tree.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                       15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                       25, 26, 27, 28, 29]})

In [39]:
y_train_pred =tree.predict(X_train) ##Predict train result
y_test_pred =tree.predict(X_test) ##Predict test result

In [40]:
 r2_score(y_train.values, y_train_pred)

0.9906048477089853

In [41]:
r2_score(y_test, y_test_pred)

0.8783316454695831

# Random Forest Regressor

In [42]:
tuned_params = {'n_estimators': [100, 200, 300, 400, 500], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
random_regressor = RandomizedSearchCV(RandomForestRegressor(), tuned_params, n_iter = 20, scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1)
random_regressor.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='neg_mean_absolute_error')

In [43]:
y_train_pred = random_regressor.predict(X_train)
y_test_pred = random_regressor.predict(X_test)

In [44]:
r2_score(y_train.values, y_train_pred)

0.9809103550134461

In [51]:
r2_score(y_test, y_test_pred)


0.887399115599125

In [53]:
xfinal= r2_score(y_test, y_test_pred)
xfinal

0.887399115599125

# Final Score of all the models

In [48]:
print("K Neighbors Regressor score  =  0.8081424233033154 ")
print("Decision Tree Regressor score =  0.8783316454695831")
print("Random Forest Regressor score =  0.887399115599125")

K Neighbors Regressor score  =  0.8081424233033154 
Decision Tree Regressor score =  0.8783316454695831
Random Forest Regressor score =  0.887399115599125


# So finaly taking Randomforest Regressor model

saving model to the disk

In [54]:
filename= 'Finalized_model.sav'
pickle.dump(xfinal,open(filename,'wb'))